# Regression

In [ ]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [ ]:
import sympy as sp
from symseaman.seaman_symbols import *
from symseaman.substitute_dynamic_symbols import run, lambdify, standard_substitutes, remove_bis, remove_bis_eq, standard_substitutes_eq, lower_and_abs
from symseaman.seaman_symbol import BisSymbol
import symseaman as ss
from symseaman.seaman_symbol import expand_bis
from symseaman.shipdict import ShipDict
import pandas as pd
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 10000)
pd.set_option('max_colwidth', -1)
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = 12, 4 
plt.style.use('bmh')
import vct.bokeh_plotting as bokeh_plotting
from numpy.testing import assert_almost_equal
from scipy.interpolate import interp1d
from symseaman.calculate import calculate_ship
from symseaman.to_matrix import matrix_form, matrix_form_standard, exclude, exclude_derivatives
from functools import reduce
from operator import add
from vct.regression_ols import Regression, RegressionPipeline
import dill

In [ ]:
# Read configs:
import anyconfig
globals_path = "../conf/base/globals.yml"
global_variables = anyconfig.load(globals_path)
ships = global_variables["ships"]
from vct.bis_system import BisSystem
import statsmodels.api as sm
from scipy.stats import norm

In [ ]:
%reload_kedro
ship_name=ships[0]
shipdict = catalog.load(f"{ship_name}.shipdict_base")
ship_dict_from_regression = catalog.load(f"{ ship_name }.ship_dict_from_regression")

df_VCT = catalog.load(f"{ship_name}.df_VCT")
df_VCT['g'] = df_VCT['g'].fillna(method='pad')
df_VCT['rho'] = df_VCT['rho'].fillna(method='pad')

thrust_keys = catalog.load(f"params:{ship_name}.thrust_keys")

if len(thrust_keys) > 0:
    df_VCT['tprop'] = df_VCT[thrust_keys[0]]
else:
    df_VCT['tprop'] = 0
    thrust_keys=['tprop']

In [ ]:
ship_name

## Regression

In [ ]:
meta_data = {
    's':shipdict.rudder_coeff_data['s'],
    #'nrud': len(shipdict.rudder_particulars),
    #'nprop':len(shipdict.rudder_particulars),
    #'xxprop':shipdict["fix_prop_data"][0]["xxfix"],
    #'tdf':shipdict["fix_prop_data"][0]["tdffix"],
    't': (shipdict.design_particulars["tf"] + shipdict.design_particulars["ta"]) / 2,
    'disp': shipdict.design_particulars["disp"],
    'lpp':  shipdict.main_data["l"],
    'rho':  df_VCT.iloc[0]["rho"],
    
    
}

In [ ]:
lpp_ = shipdict.main_data['l']
t_ = shipdict.design_particulars['ta']
disp_ = shipdict.design_particulars['disp']
units={'fy_rudders':'force',
       'fy_rudder':'force',
       'tprop':'force',
      }
bis_system = BisSystem(lpp=lpp_, volume=disp_, units=units)

interesting = list((set(bis_system.qd.keys()) & set(df_VCT.columns)) |  (set(df_VCT.columns) & set(['test type', 'model_name','fy_rudders','fy_rudder','tprop'])))
interesting = list(set(interesting))
df_bis = bis_system.df_to_bis(df_VCT[interesting])

### $ktyh$

In [ ]:
class Regression_ktyh(Regression):
    _eq = ss.equations.sway.eq_expanded.subs([
    (phi.bis,0),
    (r_w.bis,0),
    (v_w.bis,0),
    (delta,0)
    ])

### $Y_{T\delta}$

In [ ]:
class Regression_ytd(Regression):
    _eq = ss.equations.sway.eq_expanded.subs([
    (phi.bis,0),
    (r_w.bis,0),
    (v_w.bis,0),
    (Y_uudelta, 0),
    ])
    intercept=True

### $Y_{uu\delta}$

In [ ]:
class Regression_yuud(Regression):
    _eq = ss.equations.sway.eq_expanded.subs([
    (phi.bis,0),
    (r_w.bis,0),
    (v_w.bis,0),
    ])

In [ ]:
class Regression_yuv(Regression):
    _eq = ss.equations.sway.eq_expanded.subs([
    (phi.bis,0),
    (r_w.bis,0),
    (delta,0),
    (Y_uuv,0), # Note!
    ])
    
class Regression_yuv_nonlinear(Regression):
    _eq = ss.equations.sway.eq_expanded.subs([
    (phi.bis,0),
    (r_w.bis,0),
    (phi.bis,0),
    (delta,0),
    (Y_vav,0), # Note!
    ])
    
class Regression_yur(Regression):
    _eq = ss.equations.sway.eq_expanded.subs([
    (phi.bis,0),
    (v_w.bis,0),
    (phi.bis,0),
    (delta,0),
    (Y_uur,0), # Note!
    ])
    
class Regression_nuv(Regression):
    _eq = ss.equations.yaw.eq_expanded.subs([
    (phi.bis,0),
    (r_w.bis,0),
    (phi.bis,0),
    (delta,0),
    (N_uuv,0), # Note!
    ])
    
class Regression_nur(Regression):
    _eq = ss.equations.yaw.eq_expanded.subs([
    (phi.bis,0),
    (v_w.bis,0),
    (phi.bis,0),
    (delta,0),
    (N_uur,0), # Note!
    ])

In [ ]:
class Regression_xxprop(Regression):
    _eq = ss.equations.yaw.eq_expanded.subs([
    (phi.bis,0),
    (v_w.bis,0),
    (r_w.bis,0),
    (delta,0),])

class Regression_xxrud(Regression):
    _eq = ss.equations.yaw.eq_expanded.subs([
    (phi.bis,0),
    (v_w.bis,0),
    (r_w.bis,0),
    ])
    


In [ ]:
class Regression_xvv(Regression):
    _eq = ss.equations.surge.eq_expanded.subs([
    (phi.bis,0),
    #(n_prop, 0), # Note!
    #(n_rud, 0), # Note!    
    (delta,0),
    (r_w.bis,0),
    (X_res.bis,0)
    ])
    intercept=True

In [ ]:
pre_set_derivatives_0 = {
    's':shipdict.rudder_coeff_data['s'],
    'kv':shipdict.rudder_coeff_data['kv'],
    'kr':shipdict.rudder_coeff_data['kr'],
}
pipeline = RegressionPipeline(shipdict=shipdict, pre_set_derivatives=pre_set_derivatives_0)

df_ = df_bis.groupby(by='test type').get_group('self propulsion')
pipeline['ktyh'] = Regression_ktyh(df=df_)
pipeline['xxprop'] = Regression_xxprop(df=df_)

df_ = df_bis.groupby(by='test type').get_group('Thrust variation')
pipeline['ytd'] = Regression_ytd(df=df_)

df_ = df_bis.groupby(by='test type').get_group('Rudder angle')
pipeline['yuud'] = Regression_yuud(df=df_)
pipeline['xxrud'] = Regression_xxrud(df=df_)

df_ = df_bis.groupby(by='test type').get_group('Drift angle')

i=0
for V_,df__ in df_.groupby(by='V'):
    
    if len(df__) < 2:
        continue
        
    pipeline[f'yuv_{i}'] = Regression_yuv(df=df__, derivatives_suffix=f'_{i}')
    pipeline[f'nuv_{i}'] = Regression_nuv(df=df__, derivatives_suffix=f'_{i}')
    i+=1
    
df_ = df_bis.groupby(by='test type').get_group('Circle')
for i,(V_,df__) in enumerate(df_.groupby(by='V')):
    pipeline[f'yur_{i}'] = Regression_yur(df=df__, derivatives_suffix=f'_{i}')
    pipeline[f'nur_{i}'] = Regression_nur(df=df__, derivatives_suffix=f'_{i}')

In [ ]:
pipeline.keys()

In [ ]:
regression = pipeline['yuv_0']
regression.eq

In [ ]:
pipeline.fit()
pipeline.derivatives

In [ ]:
pre_set_derivatives_2 = pre_set_derivatives_0.copy()
pre_set_derivatives_2.update({
    'yvav':pipeline.derivatives['yvav'],
    'nvav':pipeline.derivatives['nvav'],
    'nrar':pipeline.derivatives['nrar'],
    #'tdf': shipdict['fix_prop_data'][0]['tdffix'],
})

pipeline2 = RegressionPipeline(shipdict=shipdict, pre_set_derivatives=pre_set_derivatives_2, 
                               pre_set_stds=pipeline.stds)

df_ = df_bis.groupby(by='test type').get_group('self propulsion')
pipeline2['ktyh'] = Regression_ktyh(df=df_)
pipeline2['xxprop'] = Regression_xxprop(df=df_)

df_ = df_bis.groupby(by='test type').get_group('Thrust variation')
pipeline2['ytd'] = Regression_ytd(df=df_)

df_ = df_bis.groupby(by='test type').get_group('Rudder angle')
pipeline2['yuud'] = Regression_yuud(df=df_)
pipeline2['xxrud'] = Regression_xxrud(df=df_)

df_ = df_bis.groupby(by='test type').get_group('Drift angle')
i=0
for V_,df__ in df_.groupby(by='V'):
    
    if len(df__) < 2:
        continue    
    
    pipeline2[f'yuv_{i}'] = Regression_yuv(df=df__, derivatives_suffix=f'_{i}')
    pipeline2[f'nuv_{i}'] = Regression_nuv(df=df__, derivatives_suffix=f'_{i}')
    pipeline2[f'xvv_{i}'] = Regression_xvv(df=df__, derivatives_suffix=f'_{i}')
    i+=1
df_ = df_bis.groupby(by='test type').get_group('Circle')
for i,(V_,df__) in enumerate(df_.groupby(by='V')):
    pipeline2[f'yur_{i}'] = Regression_yur(df=df__, derivatives_suffix=f'_{i}')
    pipeline2[f'nur_{i}'] = Regression_nur(df=df__, derivatives_suffix=f'_{i}')

In [ ]:
pipeline2.fit()
pipeline2.derivatives

In [ ]:
pipeline2.stds

In [ ]:
regression = pipeline2['xvv_1']
regression.eq

In [ ]:
regression.eq_excluded

In [ ]:
pipeline2['xvv_1'].summary()

In [ ]:
pipeline2.plot_fit();

In [ ]:
new_shipdict = pipeline2.create_shipdict()

In [ ]:
assert new_shipdict.non_lin_coeff_data['cdlever'] == pipeline.derivatives['cdlever']

In [ ]:
def assemble_derivatives(shipdict, name:str):
    s = pd.Series({
    
        'cd' : shipdict.non_lin_coeff_data['cd'],
        'cdlever' : shipdict.non_lin_coeff_data.get('cdlever',0),
        'kr' : shipdict.rudder_coeff_data['kr'],
        'kv' : shipdict.rudder_coeff_data['kv'],
        'kuv': shipdict.lin_hull_coeff_data['kuv'],
        'nur': shipdict.lin_hull_coeff_data['nur'],
        'nuur': shipdict.lin_hull_coeff_data['nuur'],
        'nuv': shipdict.lin_hull_coeff_data['nuv'],
        'nuuv': shipdict.lin_hull_coeff_data['nuuv'],
        'nuuf': shipdict.lin_hull_coeff_data['nuuf'],
        'yuv': shipdict.lin_hull_coeff_data['yuv'],
        'yuuv': shipdict.lin_hull_coeff_data['yuuv'],
        'yur': shipdict.lin_hull_coeff_data['yur'],
        'yuur': shipdict.lin_hull_coeff_data['yuur'],    
        'xrr': shipdict.resistance_data['xrr'],
        'xvr': shipdict.resistance_data['xvr'],
        'xvv': shipdict.resistance_data['xvv'],
        'yuud': shipdict.rudder_coeff_data['yuud'],
        'ytd': shipdict.rudder_coeff_data['ytd'],
        's': shipdict.rudder_coeff_data['s'],
        'xyrd': shipdict.rudder_coeff_data['xyrd'],
        'xxrud' : shipdict.rudder_particulars[0]['xxrud'],
        
    }, name=name)
    return s

In [ ]:
df_coefficients = pd.DataFrame([
    assemble_derivatives(shipdict=shipdict,name='base'),
    assemble_derivatives(shipdict=new_shipdict,name='regression'),
    
])

In [ ]:
df_coefficients

In [ ]:
ncols = 3
nrows = int(np.ceil(len(df_coefficients.columns)/ncols))
fig,axes = plt.subplots(nrows=nrows, ncols=ncols)
fig.set_size_inches(15,15)
for ax, key in zip(axes.flatten(), df_coefficients):
    
    df_coefficients[key].plot.barh(ax=ax)
    ax.set_title(key)
    
    amplitude = df_coefficients[key].max() - df_coefficients[key].min()
    ax.set_xlim(df_coefficients[key].min()-0.1*amplitude, df_coefficients[key].max()+0.1*amplitude)
    ax.grid(True)
    
for ax in axes[:,1:].flatten():
    ax.set_yticklabels([])
    
plt.tight_layout()

In [ ]:
results_sym_seaman = calculate_ship(shipdict=new_shipdict, df=df_VCT)

In [ ]:
import symseaman.calculate
symseaman.calculate.lambda_surge

In [ ]:
results = pd.concat((results_sym_seaman, df_VCT), axis=0, ignore_index=True)

In [ ]:
#ys = ['fx','fy','mz','fx_hull','fy_hull','mz_hull','fx_rudder','fy_rudder','mz_rudder']
ys = ['fx','fy','mz']
  
tabs = bokeh_plotting.create_tab(df_VCT = results, ys=ys, plot_height=300)

## Save

In [ ]:
ship_dict_from_regression['OLS'] = new_shipdict
catalog.save(f"{ ship_name }.ship_dict_from_regression", data=ship_dict_from_regression)

In [ ]:
%reload_kedro
derivatives = catalog.load(f"{ship_name}.derivatives")
derivatives_std = catalog.load(f"{ship_name}.derivatives_std")

derivatives["OLS"] = {key:float(derivative) for key,derivative in pipeline2.derivatives.items()}
derivatives_std["OLS"] = {key:float(derivative) for key,derivative in pipeline2.stds.items()}

catalog.save(f"{ship_name}.derivatives", derivatives)
catalog.save(f"{ship_name}.derivatives_std", derivatives_std)


In [ ]:
pipeline2.show()